# KOSMA-$\tau^3$

This is a working notebook to run and debug the PDR synthetic observation software KOSMA-$\tau^3$.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
import os
import sys
import inspect
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size

pathname = inspect.getframeinfo(inspect.currentframe()).filename
sys.path.append(os.path.abspath(os.path.dirname(pathname))+'/classes/')

from Model import *
import constants

The model parameters are set in this cell, with different flags available to limit the scope of model execution.

 - [x] **modelFlag**: Setup and evaluate the model emission for each voxel
 - [x] **rtFlag**: Evaluate the synthetic observation for a predefined number of sightlines
 - [ ] **cyplotFlag**: Re-grid the sightlines using `cygrid` and plot the map
 
 - **x,y,z**: overall size of model
 - **shape**: all fit within given dimensions
   - [x] block
   - [x] disk
   - [x] ellipsoid
   - [ ] shell
 - **resolution**: in parsecs
 - **focus**: how to limit the evaluated wavelengths
   - molecular: $ \lambda > 10^{-4}m $
   - PAH: $ \lambda > 5 * 10^{-6}m $
   - all
 - **modelFolder**: needed when not running the full model; the directory of the model in `$ KOSMA-tau^3/history/`

In [3]:
complete = False

modelFlag = False
rtFlag = True
cyplotFlag = True

x = 5
y = 5
z = 5

shape = 'disk'

resolution = 1000

focus = 'molecular'

constants.changeDirectory('MilkyWay')

modelFolder = 'r1000_n96/'

# Factors
constants.clumpMassFactor = 1
constants.interclumpMassFactor = 1
constants.FUVFactor = 1
constants.DensityFactor = 1

# Constant
constants.interclumpLogFUV = 1

# Model masses
constants.clumpLogMassNum = 4
constants.clumpLogMassRange = [-1, 2]
constants.interclumpLogMassNum = 2
constants.interclumpLogMassRange = [-3, -2]

In [4]:
# Limit dust calculation

print('KOSMA-tau^3')

species = ['13CO 10', 'C+ 1', 'CO 1', 'CO 2', 'CO 3', 'CO 4', 'CO 5', 'CO 6', 'CO 7', 'CO 8', 'CO 9', 'CO 10', '13CO 1', '13CO 2', '13CO 3', '13CO 4', '13CO 5', '13CO 6', '13CO 7', '13CO 8', '13CO 9', '13CO 10', 'O 2']
kosma = Model(x, y, z, modelType=shape, resolution=resolution, dustLim=focus)
kosma.addSpecies(species)

KOSMA-tau^3


In [11]:
import sys

sys.path.insert(1, "/home/craig/Desktop/Köln/KOSMA-tau^3/")

import spectrum

In [17]:
spectrum.spectrum(PAH=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Setup

In [5]:
if modelFlag:

  kosma.calculateModel()
  kosma.writeEmission()
  
  modelFolder = constants.history

## Radiative transfer

In [6]:
# Calculate integrated intensity maps

if rtFlag==False:

  import radiativeTransfer

  # radiativeTransfer.plotModel(plot='velocity', directory='r1000.0_n3015/')

  radiativeTransfer.calculateObservation(directory=modelFolder, dim='spherical', terminal=False)

### Plot intensity map

In [11]:
if rtFlag:
  plt.close('all')

  pos = fits.open('/home/yanitski/projects/pdr/KOSMA-tau^3/history/MilkyWay/r1000_n96/integrated_intensity.fits')[0]    
  intensity = fits.open('/home/yanitski/projects/pdr/KOSMA-tau^3/history/MilkyWay/r1000_n96/integrated_intensity.fits')[1]

  velocity = np.linspace(intensity.header['CRVAL4']-intensity.header['CRPIX4']*intensity.header['CDELT4'], intensity.header['CRVAL4']+intensity.header['CRPIX4']*intensity.header['CDELT4'], num=intensity.header['NAXIS4'])
  intensity_integrated = np.trapz(intensity.data, velocity, axis=0)

  if False:

    for i in range(intensity.data[:,0,0,0].size):

      proj = 'Galactic intensity map'
      fig = plt.figure(figsize=(10,10))
      ax = fig.add_subplot(111, projection='mollweide')

      # aspect = 20
      # pad_fraction = 0.5
      # divider = make_axes_locatable(ax)
      # width = axes_size.AxesY(ax, aspect=1./aspect)
      # pad = axes_size.Fraction(pad_fraction, width)
      # cax = divider.append_axes("right", size=width, pad=pad)

  #     cm = ax.scatter(pos.data[:,0], pos.data[:,1], c=intensity_integrated[:,:,23].flatten(), cmap='cubehelix', s=500, alpha = 0.5, marker='s')
      cm = ax.scatter(pos.data[:,0], pos.data[:,1], c=intensity.data[i,:,:,23].flatten(), cmap='cubehelix', s=64, alpha = 0.5, marker='s')
      cb = fig.colorbar(cm, extend='both', ax=ax, fraction = 0.02, )
      cb.ax.set_ylabel('Intensity (K km/s)', fontsize=20, labelpad=20, rotation=270)
  
  else:
    
    proj = 'Galactic intensity map'
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='mollweide')

    # aspect = 20
    # pad_fraction = 0.5
    # divider = make_axes_locatable(ax)
    # width = axes_size.AxesY(ax, aspect=1./aspect)
    # pad = axes_size.Fraction(pad_fraction, width)
    # cax = divider.append_axes("right", size=width, pad=pad)

#     cm = ax.scatter(pos.data[:,0], pos.data[:,1], c=intensity_integrated[:,:,23].flatten(), cmap='cubehelix', s=500, alpha = 0.5, marker='s')
    cm = ax.scatter(pos.data[:,0], pos.data[:,1], c=intensity_integrated[:,:,23].flatten(), cmap='cubehelix', s=64, alpha = 0.5, marker='s')
    cb = fig.colorbar(cm, extend='both', ax=ax, fraction = 0.02, )
    cb.ax.set_ylabel('Intensity (K km/s)', fontsize=20, labelpad=20, rotation=270)

  ax.set_title(proj+r' $^{13}CO^{+}$', fontsize=24)
  ax.grid(True)
  ax.set_xticklabels([])
  ax.set_yticklabels([])

  plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/yanitski/miniconda3/lib/python3.7/site-packages/matplotlib/projections/geo.py:414: RuntimeWarning: invalid value encountered in arcsin
  theta = np.arcsin(y / np.sqrt(2))


In [42]:
intensity.data.shape

(31, 50, 50, 68)

In [8]:
intensity_integrated.shape

(50, 50, 68)

In [43]:
from itertools import product, combinations

fig = plt.figure()
ax = fig.gca(projection='3d')
#ax.set_aspect("equal")

for r in [[1,0], [0,1], [0,0]]:
  for s, e in combinations(np.array(list(product(r, r, r))), 2):
      if np.sum(np.abs(s-e)) == r[1]-r[0]:
          ax.plot3D(*zip(s, e), color="k")

ax.scatter(np.random.rand(100),np.random.rand(100),np.random.rand(100), s=16*np.random.rand(100))
          
plt.show()

/home/yanitski/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
if rtFlag:
  positions = fits.open('/home/yanitski/projects/pdr/KOSMA-tau^3/history/MilkyWay/r1000_n96/voxel_position.fits')[0].data/1000.
  vel = fits.open('/home/yanitski/projects/pdr/KOSMA-tau^3/history/MilkyWay/r1000_n96/voxel_velocity.fits')[0].data

  fig = plt.figure()
  ax = fig.add_subplot(111, projection='3d')
  cm = ax.scatter(positions.T[0], positions.T[1], positions.T[2], c='k', cmap='cubehelix', marker='s', s=81, alpha=0.5, linewidth=0)
  #cb = fig.colorbar(cm)
  #cb.ax.set_ylabel(r'Radial velocity ($\frac{km}{s}$)', labelpad=16, rotation=270)

  lon,lat = np.meshgrid(np.linspace(-np.pi, np.pi, num=9), np.linspace(-np.pi/2, np.pi/2, num=5))
  x = 10 * (np.cos(lat.flatten()) * np.cos(lon.flatten()))
  y = 10 * (np.cos(lat.flatten()) * np.sin(lon.flatten()))
  z = 10 * np.sin(lat.flatten())

  #ax.scatter([8.178], [0], [0], marker='o', s=49, c='b')
  #ax.quiver(np.full_like(x, 8.178), np.full_like(x, 0), np.full_like(x, 0), x, y, z, color='k', arrow_length_ratio=0.2)

  ax.set_xlabel('X (kpc)')
  ax.set_ylabel('Y (kpc)')
  ax.set_zlabel('Z (kpc)')
  ax.set_xlim([-6, 6])
  ax.set_ylim([-6, 6])
  ax.set_zlim([-6, 6])
  plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
if rtFlag:
  plt.close()

  positions = fits.open('/home/yanitski/projects/pdr/KOSMA-tau^3/history/MilkyWay/r1000_n96/voxel_position.fits')[0].data/1000.
  vel = fits.open('/home/yanitski/projects/pdr/KOSMA-tau^3/history/MilkyWay/r1000_n96/voxel_velocity.fits')[0].data

  idx_plane = positions[:,2]==0
  idx_Earth = (np.sqrt(positions[:,0]**2+positions[:,1]**2)<=8.178) & (positions[:,0]>=0) & idx_plane

  idx = idx_Earth
  idx_obs = []

  for i in range(18,-19,-1):
      line = ''
      iy = i==positions[idx,1]
      orbit = True
      for j in range(18,-19,-1):
          ix = j==positions[idx,0]
          if (ix&iy).any() and orbit:
              line += 'XX'
              idx_obs.append(np.where(ix&iy)[0])
              orbit = False
          else: line += '--'
      #print(line)

  idx_obs = np.array(idx_obs)

  fig = plt.figure()
  ax = fig.add_subplot(111, projection='3d')
  cm = ax.scatter(positions.T[0], positions.T[1], positions.T[2], c=vel[:,0], cmap='cubehelix', marker='s', s=27, alpha=0.5, linewidth=0)
  cb = fig.colorbar(cm)
  cb.ax.set_ylabel(r'Radial velocity ($\frac{km}{s}$)', labelpad=16, rotation=270)

  # lon,lat = np.meshgrid(np.linspace(-np.pi, np.pi, num=9), np.linspace(-np.pi/2, np.pi/2, num=5))
  # x = 10 * (np.cos(lat.flatten()) * np.cos(lon.flatten()))
  # y = 10 * (np.cos(lat.flatten()) * np.sin(lon.flatten()))
  # z = 10 * np.sin(lat.flatten())
  #print(idx_obs)
  ax.scatter(positions[idx,0][idx_obs], positions[idx,1][idx_obs], positions[idx,2][idx_obs], marker='o', s=49, c='b')
  ax.quiver(positions[idx,0][idx_obs], positions[idx,1][idx_obs], positions[idx,2][idx_obs], np.full_like(idx_obs, -24.178*np.sin(positions[idx,0][idx_obs]/8.178/2*np.pi)), 0,0, color='k')
  ax.quiver(positions[idx,0][idx_obs], positions[idx,1][idx_obs], positions[idx,2][idx_obs], np.full_like(idx_obs, 10), 0,0, color='k')

  ax.set_xlabel('X (kpc)')
  ax.set_ylabel('Y (kpc)')
  ax.set_zlabel('Z (kpc)')
  ax.set_xlim([-18, 18])
  ax.set_ylim([-18, 18])
  ax.set_zlim([-18, 18])
  plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IndexError: arrays used as indices must be of integer (or boolean) type

## Cygrid

In [ ]:
# Create cygrid images

if cyplotFlag:

  constants.mapShape = (200, 200)

  import cyplot

  images,wcs = cyplot.convertMap(modelFolder, input_coord='spherical')
  print(images.shape)

  images = np.nan_to_num(images)

  vmin = images[0,1,:,:].min()
  vmax = images[0,1,:,:].max()

  fig = plt.figure()
  ax1 = fig.add_subplot(111, projection=wcs.celestial)
  dust = ax1.imshow(images[0,0,:,:], alpha=0.5, vmin=vmin, vmax=vmax, cmap='bone')
  cbar1 = plt.colorbar(dust, ax=ax1, shrink=1)
  cbar1.ax.set_ylabel('Dust Intensity (K km/s)', rotation=270)
#   i = 23
#   mol1 = ax1.imshow(image[0,i,:,:], alpha=0.5, cmap='magma')
#   cbar2 = plt.colorbar(mol1, ax=ax1, shrink=1)
#   cbar2.ax.set_ylabel('{} Intensity (K km/s)'.format(species[i-333]), rotation=270)
#   i = 44
#   mol2 = ax1.imshow(image[0,i,:,:], alpha=0.5, cmap='hot')
#   cbar3 = plt.colorbar(mol2, ax=ax1, shrink=1)
#   cbar3.ax.set_ylabel('{} Intensity (K km/s)'.format(species[i-333]), rotation=270)
  
  plt.show()